## Generalized ufuncs

We've just seen how to make our own ufuncs using `vectorize`, but what if we need something that can operate on an arbitrary number of elements in an input array?  

Enter `guvectorize`.  

There are several important differences between `vectorize` and `guvectorize` that bear close examination.  Let's take a look at a few simple examples.

In [ ]:
import numpy
from numba import guvectorize

In [ ]:
@guvectorize(['(int64[:], int64, int64[:])'], '(n),()->(n)')
def g(x, y, res):
    for i in range(x.shape[0]):
        res[i] = x[i] + y

* Declaration of input/output layouts
* No return statements

In [ ]:
x = numpy.arange(10)
res = numpy.empty_like(x)

In [ ]:
g(x, 5, res)

In [ ]:
res

In [ ]:
@guvectorize(['float64[:,:], float64[:,:], float64[:,:]'], 
            '(m,n),(n,p)->(n,p)')
def matmul(A, B, C):
    m, n = A.shape
    n, p = B.shape
    for i in range(m):
        for j in range(p):
            C[i, j] = 0
            for k in range(n):
                C[i, j] += A[i, k] * B[k, j]

In [ ]:
a = numpy.random.random((500, 500))

In [ ]:
%timeit matmul(a, a, numpy.empty_like(a))

In [ ]:
%timeit a @ a

## [Exercise: Cheating input/output layouts](./exercises/08.GUVectorize.Exercises.ipynb#Exercise:-Cheating-input/output-layouts)

In [ ]:
# %load snippets/guvectorize/cheatio.py

## [Exercise: Multigrid Restriction](./exercises/08.GUVectorize.Exercises.ipynb#Exercise:-Multigrid-Restriction)

In 2D, the full weighting restriction operator is given as

\begin{align}
v_{(i,j),c} &=\frac{1}{16} \left(v_{(2i-1,2j-1),f} +v_{(2i-1,2j+1),f} +v_{(2i+1,2j-1),f}  +v_{(2i+1,2j+1),f} \right)\notag \\
&\ + \frac{1}{8} \left(v_{(2i,2j-1),f} + v_{(2i,2j+1),f} +v_{(2i-1,2j),f}  +v_{(2i+1,2j),f} \right) \\
&\ + \frac{1}{4} v_{(2i,2j),f} \\
& \texttt{ for } \left\lbrace 1 \le i,j \le N_{x,c}-2 \right.
\end{align}


In [ ]:
# %load snippets/guvectorize/restriction.py

In [ ]:
fine = numpy.arange(100).reshape(10,10)
coarse = numpy.zeros((5, 5))
end = restrict_2d_gvec(fine, numpy.empty(coarse.shape[0]), coarse)
end

In [ ]:
from matplotlib import pyplot, cm
%matplotlib inline

In [ ]:
from matplotlib import cbook

In [ ]:
filename = cbook.get_sample_data('jacksboro_fault_dem.npz', asfileobj=False)
with numpy.load(filename) as dem:
    z = dem['elevation']

In [ ]:
z = z[:,:344]

In [ ]:
z = numpy.float32(z)
z.dtype

In [ ]:
zorig = z.copy()

In [ ]:
%%timeit -r 5
y = numpy.zeros_like(z[::2,::2])
restrict_2d_gvec(z, numpy.empty(y.shape[0]), y)

In [ ]:
pyplot.figure(figsize=(10,10))
pyplot.imshow(z, interpolation='none', cmap=cm.viridis)
pyplot.colorbar();

In [ ]:
pyplot.figure(figsize=(10,10))
y = numpy.copy(z[::2,::2])
restrict_2d_gvec(z, y, y)
pyplot.imshow(y, interpolation='none', cmap=cm.viridis)
pyplot.colorbar()
z = y.copy()

### We can destroy information!  What now?

## [Exercise: Interpolation in 2D](./exercises/08.GUVectorize.Exercises.ipynb#Exercise:-Multigrid-Interpolation)

The **interpolation in 2D** is

\begin{align}
v_{(2i,2j),f} &= v_{(i,j),c} \texttt{ for } \left\{ 0 \le i,j \le N_{x,c}-1 \right.\\
v_{(2i+1,2j),f} &= \frac{1}{2}\left(v_{(i,j),c}+v_{(i+1,j),c} \right) 
\texttt{ for }  \left\lbrace \begin{matrix} 0 \le i \le N_{x,c}-2 \\ 0 \le j \le N_{x,c}-1 \end{matrix} \right .\\
v_{(2i,2j+1),f} &= \frac{1}{2}\left(v_{(i,j),c}+v_{(i,j+1),c} \right) \texttt{ for } \left\lbrace \begin{matrix} 0 \le i \le N_{x,c}-1 \\ 0 \le j \le N_{x,c}-2 \end{matrix} \right.\\
v_{(2i+1,2j+1),f} &= \frac{1}{4}\left(v_{(i,j),c}+v_{(i+1,j),c}+v_{(i,j+1),c}+v_{(i+1,j+1),c}\right) \texttt{ for } \left\lbrace 0 \le i,j \le N_{x,c}-2 \right.
\end{align}

In [ ]:
# %load snippets/guvectorize/interpolation.py

In [ ]:
y = z

In [ ]:
x = numpy.ones((y.shape[0]*2, y.shape[1]*2)) * y.min()
interpolate_2d_gvec(y, numpy.empty(x.shape[0]), x)
y = x.copy()

pyplot.figure(figsize=(8,8))
pyplot.imshow(x, interpolation='none', cmap=cm.viridis)
pyplot.colorbar();

In [ ]:
%%timeit
x = numpy.zeros((y.shape[0]*2, y.shape[1]*2))
interpolate_2d_gvec(y, numpy.empty(x.shape[0]), x)

## Larger arrays

In [ ]:
z = numpy.random.random((10000, 10000))

In [ ]:
%%timeit
y = numpy.zeros_like(z[::2,::2])
restrict_2d_gvec(z, numpy.empty(y.shape[0]), y)